In [9]:
library(tidyverse)
# install.packages("caret")
library(caret)
# install.packages("randomForest")
library(randomForest)
library(xgboost)
library(mltools)
library(data.table)
library(mlbench)
library(caret)

# Preprocessing

In [10]:
wine <- read.csv("data/train.csv")
head(wine,5)

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,type
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
1,6.6,0.3,0.36,1.2,0.035,43,126,0.9909,3.01,0.63,11.4,6,white
2,7.7,0.5,0.26,1.9,0.062,9,31,0.9966,3.39,0.64,9.6,5,red
3,8.4,0.5,0.35,2.9,0.076,21,127,0.9976,3.23,0.63,9.2,5,red
4,7.5,0.4,0.33,5.0,0.045,30,131,0.9942,3.32,0.44,10.9,6,white
5,6.4,0.2,0.25,20.2,0.083,35,157,0.9998,3.17,0.50,9.1,5,white


In [11]:
#Transforming Quality from an Integer to a Factor
wine$quality <- factor(wine$quality, ordered = T)

# Creating a new Factored Variable called 'Rating'

wine$rating <- ifelse(wine$quality < 5, 'bad', ifelse(
  wine$quality < 8, 'average', 'good'))

wine$rating <- ordered(wine$rating,
                       levels = c('bad', 'average', 'good'))

In [12]:
wine$type <- as.factor(wine$type)
wine <- one_hot(as.data.table(wine))

In [13]:
freq_count  <-  table(wine$rating)
wine$rating    <-  match(wine$rating, names(freq_count))

In [14]:
summary(wine)


 fixed.acidity    volatile.acidity  citric.acid     residual.sugar  
 Min.   : 3.800   Min.   :0.1000   Min.   :0.0000   Min.   : 0.600  
 1st Qu.: 6.400   1st Qu.:0.2000   1st Qu.:0.2400   1st Qu.: 1.800  
 Median : 7.000   Median :0.3000   Median :0.3100   Median : 3.000  
 Mean   : 7.252   Mean   :0.3484   Mean   :0.3175   Mean   : 5.412  
 3rd Qu.: 7.700   3rd Qu.:0.4000   3rd Qu.:0.3900   3rd Qu.: 8.000  
 Max.   :15.900   Max.   :1.3000   Max.   :1.6600   Max.   :65.800  
                                                                    
   chlorides       free.sulfur.dioxide total.sulfur.dioxide    density      
 Min.   :0.00900   Min.   :  1.00      Min.   :  6.0        Min.   :0.9871  
 1st Qu.:0.03800   1st Qu.: 16.00      1st Qu.: 75.0        1st Qu.:0.9924  
 Median :0.04800   Median : 28.00      Median :117.0        Median :0.9951  
 Mean   :0.05751   Mean   : 30.13      Mean   :114.6        Mean   :0.9948  
 3rd Qu.:0.06800   3rd Qu.: 41.00      3rd Qu.:155.0        3rd

In [15]:
process <- preProcess(as.data.frame(wine), method=c("range"))

wine <- predict(process, as.data.frame(wine))
head(wine,5)

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,type_red,type_white,rating
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<ord>,<dbl>,<dbl>,<dbl>
1,0.2314050,0.16666667,0.2168675,0.009202454,0.04318937,0.14583333,0.27649770,0.07321773,0.2248062,0.2303371,0.4927536,6,0,1,0.5
2,0.3223140,0.33333333,0.1566265,0.019938650,0.08803987,0.02777778,0.05760369,0.18304432,0.5193798,0.2359551,0.2318841,5,1,0,0.5
3,0.3801653,0.33333333,0.2108434,0.035276074,0.11129568,0.06944444,0.27880184,0.20231214,0.3953488,0.2303371,0.1739130,5,1,0,0.5
4,0.3057851,0.25000000,0.1987952,0.067484663,0.05980066,0.10069444,0.28801843,0.13680154,0.4651163,0.1235955,0.4202899,6,0,1,0.5
5,0.2148760,0.08333333,0.1506024,0.300613497,0.12292359,0.11805556,0.34792627,0.24470135,0.3488372,0.1573034,0.1594203,5,0,1,0.5


In [ ]:

# Train-Validation Split

trainIndex <- createDataPartition(wine$quality, p = .8, 
                                  list = FALSE,
                                  times = 1)
trainData <- wine[trainIndex,]

validationData <- trainData[-trainIndex,]
seed <- 42
set.seed(seed)  # For reproducibility
control <- trainControl(method="repeatedcv", number=5, repeats=3, search="grid")
tunegrid <- expand.grid(.mtry=c(1:15))
model <- train(quality~., data=wine, method="rf", tuneGrid=tunegrid, trControl=control)
print(model)
plot(model)



Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
“invalid mtry: reset to within valid range”
Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
“invalid mtry: reset to within valid range”
Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
“invalid mtry: reset to within valid range”
Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
“invalid mtry: reset to within valid range”
Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
“invalid mtry: reset to within valid range”
Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
“invalid mtry: reset to within valid range”
Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
“invalid mtry: reset to within valid range”
Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
“invalid mtry: reset to within valid range”
Warning message in randomForest.default(x, y, mtry = param$mtry, ...):
“invalid 

In [ ]:
# predictions <- predict(model, newdata = validationData)

# # Calculate RMSE
# rmse <- sqrt(mean((predictions - validationData$quality)^2))
# print(paste("RMSE:", rmse))

[1] "RMSE: 0.258755579459304"




In [ ]:
# # Split the data into training and validation sets
# trainData <- read.csv("/kaggle/input/dsr302m/train.csv")
# testData <- read.csv("/kaggle/input/dsr302m/test.csv")

# # Train-Validation Split
# trainIndex <- createDataPartition(trainData$quality, p = .8, 
#                                   list = FALSE,
#                                   times = 1)
# trainData <- trainData[ trainIndex,]
# validationData <- trainData[-trainIndex,]

# #                  eta = 0.1, # Learning rate
# #                  max_depth = 6, 
# #                  nrounds = 500, # Number of estimators
# #                  objective = "reg:squarederror", 
# #                  subsample = 0.5,
# #                  colsample_bytree = 0.5,
# #                  seed = 42)

# # Train a Random Forest Model
# model <- xgboost(data = trainData, 
#                  eta = 0.09, # Learning rate
#                  max_depth = 12, 
#                  objective = "reg:squarederror", 
#                  seed = 42)
# # Make predictions on the validation data
# predictions <- predict(model, newdata = validationData)

# # Calculate RMSE
# rmse <- sqrt(mean((predictions - validationData$quality)^2))
# print(paste("RMSE:", rmse))


In [ ]:
# # Train the final model on the entire training data
# final_model <- randomForest(quality ~ ., data = trainData, ntree = 100)

# # Make predictions on the test data
# test_predictions <- predict(final_model, newdata = testData)
# rounded_predictions <- round(test_predictions)
# # Create a data frame with 'id' and 'quality' columns
# result <- data.frame(id = testData$id, quality = rounded_predictions)

# # Save the results to a CSV file
# write.csv(result, "result.csv", row.names = FALSE)


In [ ]:
# result_data <-read.csv("/kaggle/working/result.csv")
# head(result_data,5)

In [ ]:

# # Load the data
# trainData <- read.csv("/kaggle/input/dsr302m/train.csv")
# testData <- read.csv("/kaggle/input/dsr302m/test.csv")

# # Feature Engineering
# # You can create new features or transform existing ones here
# # Example: Creating a feature for the ratio of alcohol to volatile acidity
# trainData$alcohol_to_volatile_acidity <- trainData$alcohol / trainData$volatile.acidity
# testData$alcohol_to_volatile_acidity <- testData$alcohol / testData$volatile.acidity

# # Data Preprocessing
# # Standardize or normalize numeric features
# numeric_features <- c("alcohol","volatile acidity")  # Add other numeric features
# trainData[numeric_features] <- scale(trainData[numeric_features])
# testData[numeric_features] <- scale(testData[numeric_features])

# # Hyperparameter Tuning
# set.seed(123)  # For reproducibility
# # You can perform hyperparameter tuning for the Random Forest model here
# tuned_model <- train(quality ~ ., data = trainData, method = "rf",
#                      trControl = trainControl(method = "cv", number = 5))

# # Get the best model
# best_model <- tuned_model$finalModel

# # Make predictions on the test data
# test_predictions <- predict(best_model, newdata = testData)

# # Round the predictions to the nearest whole number
# rounded_predictions <- round(test_predictions)

# # Create a data frame with 'id' and 'quality' columns
# result <- data.frame(id = testData$id, quality = rounded_predictions)

# # Save the results to a CSV file
# write.csv(result, "result.csv", row.names = FALSE)
